In [1]:
## Import `Sequential` from `keras.models`
from tensorflow.keras.models import Sequential

# Import `Dense` from `keras.layers`
from tensorflow.keras.layers import Dense

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf

In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.metrics import roc_auc_score, auc, accuracy_score, confusion_matrix, mean_squared_error, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split, StratifiedKFold

import xgboost as xgb
import multiprocessing

import warnings
warnings.filterwarnings('ignore')

In [ ]:
test_df = pd.read_csv('test_result_complete.csv', parse_dates = ['date'])
test_df.head(10)

In [ ]:
train_df = pd.read_csv('train_result_complete.csv', parse_dates = ['date'])
train_df.head(10)

In [ ]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [ ]:
options = ['ATOM-USD', 'BTC-USD', 'DOGE-USD', 'DOT-USD', 'ETH-USD', 'ETC-USD', 'POLY-USD', 'SHIB-USD'] 
# selecting rows based on condition 
sample_df = train_df[train_df['coinpair'].isin(options)]
sample_test_df = test_df[test_df['coinpair'].isin(options)] 

In [ ]:
train_X = sample_df[['x_rsi',
       'MA_1_3', 'MA_3_24', 'MA_24_216', 'x_ATR', 'x_bollinger',
       'obv', 'MA_3_8', 'MA_8_24', 'MA_24_48', 'coinpair']]
train_y = sample_df['y_4']

In [ ]:
test_X = sample_test_df[['x_rsi',
        'MA_1_3', 'MA_3_24', 'MA_24_216', 'x_ATR', 'x_bollinger',
        'obv', 'MA_3_8', 'MA_8_24', 'MA_24_48', 'coinpair']]
test_y = sample_test_df['y_4']

In [ ]:
transformer = make_column_transformer(
        (OneHotEncoder(), ['coinpair']),
        remainder='passthrough')

transformed = transformer.fit_transform(train_X)

In [ ]:
train_X = pd.DataFrame(
    transformed, 
    columns=transformer.get_feature_names()
)

In [ ]:
test_transformed = transformer.transform(test_X)

In [ ]:
test_X = pd.DataFrame(
    test_transformed, 
    columns=transformer.get_feature_names()
)

In [ ]:
# normalizer = tf.keras.layers.Normalization(axis=-1)
# normalizer.adapt(train_X)
# (value - min)/(max - min)

In [ ]:
# def create_model(activation = 'relu', hidden_layers=1, units=6):
#     # Initialize the constructor
#     model = Sequential()
#     # Add an input layer
#     model.add(normalizer)
#     # model.add(Dense(train_X.shape[1], activation=activation, input_shape=(train_X.shape[1],)))

#     for i in range(hidden_layers):
#     # Add one hidden layer
#         model.add(Dense(units, activation=activation))

#     # Add an output layer 
#     model.add(Dense(3, activation='softmax'))
#     #compile model
#     model.compile(loss="categorical_crossentropy", optimizer= 'adam', metrics=['accuracy',tf.keras.metrics.AUC(multi_label=True)])
#     return model

In [ ]:
# my_classifier = KerasClassifier(build_fn=create_model)# Create 
# parameters = {'hidden_layers' : [3, 5], 'units' : [5, 10], 'activation' : ['relu','sigmoid']}
# # Create grid search
# grid = RandomizedSearchCV(estimator=my_classifier, param_distributions=parameters) #inserir param_distributions

# # Fit grid search
# grid_result = grid.fit(train_X, train_y,
#                     verbose=1)

In [ ]:
# results = pd.DataFrame(grid_result.cv_results_)
# results.to_csv('/content/drive/MyDrive/nn-random-grid-search-results-01.csv', index=False)

In [ ]:
# View hyperparameters of best neural network
# grid_result.best_params_

In [ ]:
# y_preb_probs = grid_result.predict_proba(test_X)
# roc_auc_score(test_y, y_preb_probs, average="weighted", multi_class="ovr")

In [ ]:
# y_preb_probs_train = grid_result.predict_proba(train_X)
# roc_auc_score(train_y, y_preb_probs_train, average="weighted", multi_class="ovr")

In [ ]:
import joblib

In [ ]:
#save your model or results
# joblib.dump(grid_result.best_estimator_, '/content/drive/MyDrive/nn_model.pkl')

#load your model for further usage
loaded_model = joblib.load("/content/drive/MyDrive/nn_model.pkl")

In [ ]:
y_preb_probs = loaded_model.predict_proba(test_X)
roc_auc_score(test_y, y_preb_probs, average="weighted", multi_class="ovr")

In [ ]:
y_preb_probs_train = loaded_model.predict_proba(train_X)
roc_auc_score(train_y, y_preb_probs_train, average="weighted", multi_class="ovr")

In [ ]:
y_pred_test = loaded_model.predict(test_X)
accuracy_score(test_y, y_pred_test)

## SUMMARY STATS OF RAW DATA

In [ ]:
sample_df[['low', 'high', 'close', 'volume']].describe()

In [ ]:
sample_df.coinpair.value_counts()/sample_df.shape[0]*100

In [ ]:
sample_test_df[['low', 'high', 'close', 'volume']].describe()

In [ ]:
sample_test_df.coinpair.value_counts()/sample_test_df.shape[0]*100

In [ ]:
sample_df[['x_rsi','x_ATR', 'obv']].describe()

In [ ]:
for each in ['MA_1_3', 'MA_3_24', 'MA_24_216', 'x_bollinger',
             'MA_3_8', 'MA_8_24', 'MA_24_48', 'coinpair']:
  print(sample_df[each].value_counts()/sample_df.shape[0]*100)

In [ ]:
sample_test_df[['x_rsi','x_ATR', 'obv']].describe()

In [ ]:
for each in ['MA_1_3', 'MA_3_24', 'MA_24_216', 'x_bollinger',
             'MA_3_8', 'MA_8_24', 'MA_24_48', 'coinpair']:
  print(sample_test_df[each].value_counts()/sample_test_df.shape[0]*100)

In [ ]:
#Summary stats of output
print(sample_test_df['y_4'].value_counts()/sample_test_df.shape[0]*100)

In [ ]:
#Summary stats of output
print(sample_df['y_4'].value_counts()/sample_df.shape[0]*100)

In [ ]:
sample_df['y_pred'] = loaded_model.predict(train_X)
sample_test_df['y_pred'] = loaded_model.predict(test_X)

In [ ]:
for each in sample_df.coinpair.unique():
    temp = sample_df.loc[sample_df.coinpair==each].copy()
    temp.reset_index(inplace = True)
    volume = 0
    value = 0
    transactions = 0
    for i in range(len(temp)):
        if temp.loc[i].y_pred == 0:
            if volume > 0:
                value += temp.loc[i].close
                transactions += 0.01*temp.loc[i].close
                volume -= 1
        elif temp.loc[i].y_pred == 2:
            continue
        else:
            volume += 1
            value -= temp.loc[i].close
            transactions += 0.01*temp.loc[i].close
            for j in range(i+1,i+48):
                if j < len(temp):
                    if temp.loc[j].y_pred == 0:
                        value += temp.loc[j].close
                        transactions += 0.01*temp.loc[j].close
                        volume -= 1
                        break
                else:
                    value += temp.loc[j-1].close
                    transactions += 0.01*temp.loc[j-1].close
                    volume -= 1
                    break
            if volume == 1:
                value += temp.loc[i+48].close
                transactions += 0.01*temp.loc[i+48].close
                volume -= 1
    print(each)
    print(value)
    print(transactions)
    print('------------------------')


In [ ]:
for each in sample_test_df.coinpair.unique():
    temp = sample_test_df.loc[sample_test_df.coinpair==each].copy()
    temp.reset_index(inplace = True)
    volume = 0
    value = 0
    transactions = 0
    for i in range(len(temp)):
        if temp.loc[i].y_pred == 0:
            if volume > 0:
                value += temp.loc[i].close
                    transactions += 0.01*temp.loc[i].close
                    volume -= 1
            elif temp.loc[i].y_pred = 2:
                continue
            else:
                volume += 1
                value -= temp.loc[i].close
                transactions += 0.01*temp.loc[i].close
                for j in range(i+1,i+48):
                    if j < len(temp):
                        if temp.loc[j].y_pred == 0:
                            value += temp.loc[j].close
                            transactions += 0.01*temp.loc[j].close
                            volume -= 1
                            break
                    else:
                        value += temp.loc[j-1].close
                        transactions += 0.01*temp.loc[j-1].close
                        volume -= 1
                        break
                if volume == 1:
                    if i+48 < len(temp):
                        value += temp.loc[i+48].close
                        transactions += 0.01*temp.loc[i+48].close
                        volume -= 1
                    else:
                        value += temp.loc[len(temp)-1].close
                        transactions += 0.01*temp.loc[len(temp)-1].close
                        volume -= 1
        print(each)
        print(value)
        print(transactions)
        print('------------------------')